In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kcc_raw.csv to kcc_raw.csv


In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
df = pd.read_csv("kcc_raw.csv")

In [ ]:
df["QueryText"] = df["QueryText"].str.strip()
df["KccAns"] = df["KccAns"].str.strip()

In [ ]:
qa_chunks = []

In [ ]:
for _, row in df.iterrows():
    text = f"Q: {row['QueryText']} A: {row['KccAns']}"
    metadata = {
        "state": row.get("StateName", ""),
        "crop": row.get("Crop", ""),
        "category": row.get("Category", ""),
        "query_type": row.get("QueryType", ""),
        "month": row.get("month", ""),
        "year": row.get("year", "")
    }
    qa_chunks.append({"text": text, "metadata": metadata})

In [ ]:
with open("kcc_chunks.json", "w", encoding="utf-8") as f:
    json.dump(qa_chunks, f, indent=2, ensure_ascii=False)

In [ ]:
print(f"✅ Saved {len(qa_chunks)} Q&A pairs to kcc_chunks.json")

✅ Saved 36671 Q&A pairs to kcc_chunks.json


In [ ]:
files.download("kcc_chunks.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.8 MB/s eta 0:

In [ ]:
import json
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

In [ ]:
with open("kcc_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
texts = [item["text"] for item in data]
metadatas = [item["metadata"] for item in data]
ids = [f"id_{i}" for i in range(len(data))]

print(f"📦 Total entries to embed: {len(texts)}")

📦 Total entries to embed: 36671


In [ ]:
print("🔄 Generating embeddings...")
model = SentenceTransformer("all-MiniLM-L6-v2")

🔄 Generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(
    texts,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
).tolist()

Batches:   0%|          | 0/2292 [00:00<?, ?it/s]

In [ ]:
print("🧠 Storing embeddings in ChromaDB...")
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.create_collection(name="kcc_qa_collection")

🧠 Storing embeddings in ChromaDB...


In [ ]:
BATCH_SIZE = 5000
for i in range(0, len(texts), BATCH_SIZE):
    collection.add(
        documents=texts[i:i+BATCH_SIZE],
        embeddings=embeddings[i:i+BATCH_SIZE],
        metadatas=metadatas[i:i+BATCH_SIZE],
        ids=ids[i:i+BATCH_SIZE]
    )
    print(f"✅ Added batch {i // BATCH_SIZE + 1}")

✅ Added batch 1
✅ Added batch 2
✅ Added batch 3
✅ Added batch 4
✅ Added batch 5
✅ Added batch 6
✅ Added batch 7
✅ Added batch 8


In [ ]:
import json
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import requests

In [ ]:
client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.get_collection(name="kcc_qa_collection")

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
user_query = input("🔍 Enter your agricultural query: ")

🔍 Enter your agricultural query: What are the recommended treatments for leaf spot disease in groundnut?


In [ ]:
query_embedding = model.encode(user_query).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=3)

In [ ]:
retrieved_docs = results['documents'][0]
retrieved_context = "\n".join(retrieved_docs)